In [7]:
import pandas as pd
import numpy as np
import time
from tqdm import tqdm

import nltk
nltk.download('stopwords')

from milvus import default_server, debug_server
from pymilvus import connections, utility, DataType, FieldSchema, CollectionSchema, Collection

from utils import generate_embeddings

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mirandadrummond/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
# Variable declaration
COLLECTION_NAME = "arxiv_10000"
DIMENSION = 1024
BATCH_SIZE = 128
TOPK = 5
COUNT = 10000

print('loading data')

loading data


In [9]:
arxiv_e = pd.read_csv('../bert/final_data.csv')
def convert(data_string):
    # Remove square brackets and split the string into individual numbers
    clean_string = data_string.replace('[', '').replace(']', '').strip()
    data_list = [float(item) for item in clean_string.split()]

    # Create a NumPy array from the list of floats
    data_array = np.array(data_list)

    return data_array

arxiv_e['embeddings'] = arxiv_e['embeddings'].apply(convert)

In [15]:
default_server.stop()

In [ ]:
connections.connect(host = "127.0.0.1", port = default_server.listen_port)

KeyboardInterrupt: 

In [ ]:
utility.get_server_version()

# Check if the collection is already available, if yes, then drop it and create a new one
if utility.has_collection(COLLECTION_NAME):
    utility.drop_collection(COLLECTION_NAME)

In [ ]:
# object should be inserted in the format of (title, date, location, speech embedding)
fields = [
    FieldSchema(name = "id", dtype = DataType.INT64, is_primary = True, auto_id = True),
    FieldSchema(name = "title", dtype = DataType.VARCHAR, max_length = 800),
    FieldSchema(name = "abstract", dtype = DataType.VARCHAR, max_length = 9000),
    FieldSchema(name = "label", dtype = DataType.VARCHAR, max_length = 20),
    FieldSchema(name = "embedding", dtype = DataType.FLOAT_VECTOR, dim = DIMENSION)
]
schema = CollectionSchema(fields = fields)
collection = Collection(name = COLLECTION_NAME, schema = schema)

# Create the index
index_params = {
    "index_type": "IVF_FLAT",
    "metric_type": "L2",
    "params": {"nlist": 100},
}
collection.create_index(field_name = "embedding", index_params = index_params)
collection.load()

In [ ]:
for batch in tqdm(np.array_split(arxiv_e, (COUNT//BATCH_SIZE) + 1), desc="Processing batches"):
    data = [
        batch['title'].tolist(),
        batch['abstract'].tolist(),
        batch['label'].tolist(),
        batch['embeddings'].tolist()
    ]

    collection.insert(data)

collection.flush()

In [ ]:
def search_similar_texts(query_text, TOPK=5):
    query_text = ["What papers talk about astrophysics?", "What are the papers on that discuss computer architecture?"]

    search_data = generate_embeddings(query_text)

    start = time.time()
    res = collection.search(
        data = search_data,  # Embed search value
        anns_field = "embedding",  # Search across embeddings
        param = {"metric_type": "L2",
                "params": {"nprobe": 20}},
        limit = TOPK,  # Limit to top_k results per search
        output_fields = ["title","abstract"]  # Include title field in result 
    )

    end = time.time()

    for hits_i, hits in enumerate(res):
        print("Query:", query_text[hits_i])
        #print("Abstract:", search_terms[hits_i])
        print("Search Time:", end-start)
        print("Results:\n")
        for hit in hits:
            print( hit.entity.get("title"), "----", round(hit.distance, 3))
            print()
            print( hit.entity.get("abstract"), "----", round(hit.distance, 3))
            print()
        print()

    return res